This is problem set #4, in which we want you to integrate your knowledge of data wrangling with some basic simulation skills. It's a short problem set to help you get your feet wet in testing statistical concepts through "making up data" rather than consulting a textbook or doing math. 

For ease of reading, please separate your answers from our text by marking our text with the `>` character (indicating quotes). 

```{r, warning=F, message=F}
library(tidyverse)
```

In [1]:
import pandas as pd
import numpy as np
import plotnine as gg
from scipy import stats


Let's start by convincing ourselves that t-tests have the appropriate false positive rate. Run 10,000 t-tests with standard, normally-distributed data from a made up 30-person, single-measurement experiment (the command for sampling from a normal distribution is `rnorm`).

The goal of these t-tests are to determine, based on 30 observations, whether the underlying distribution (in this case a normal distribution with mean 0 and standard deviation 1) has a mean that is different from 0. In reality, the mean is not different from 0 (we sampled it using `rnorm`), but sometimes the 30 observations we get in our experiment will suggest that the mean is higher or lower. In this case, we'll get a "significant" result and incorrectly reject the null hypothesis of mean 0.

What's the proportion of "significant" results ($p < .05$) that you see?

First do this using a `for` loop.

```{r}
```

In [12]:
significant_count = 0
for _ in range(30):
    dist = np.random.normal(size=30)
    ttest_result = stats.ttest_1samp(dist, 0)
    print(ttest_result)
    if ttest_result.pvalue < 0.05:
        significant_count += 1

print(significant_count)

Ttest_1sampResult(statistic=-0.07094421989346486, pvalue=0.943929144365294)
Ttest_1sampResult(statistic=-0.021255965221569362, pvalue=0.9831870514014316)
Ttest_1sampResult(statistic=0.5687950917186169, pvalue=0.5738742137333928)
Ttest_1sampResult(statistic=-0.3794827792177964, pvalue=0.7070941174982196)
Ttest_1sampResult(statistic=-1.7738250301943719, pvalue=0.08659420086299342)
Ttest_1sampResult(statistic=-1.169002920951274, pvalue=0.2519220325901182)
Ttest_1sampResult(statistic=0.8449911299115621, pvalue=0.4050334626947413)
Ttest_1sampResult(statistic=-0.790046223145534, pvalue=0.4359137168633419)
Ttest_1sampResult(statistic=-0.08706178465196417, pvalue=0.9312207823824327)
Ttest_1sampResult(statistic=-0.36252301889292265, pvalue=0.7195899648637516)
Ttest_1sampResult(statistic=1.9424784786904041, pvalue=0.06184644937765668)
Ttest_1sampResult(statistic=1.6353961528868861, pvalue=0.11277562225635614)
Ttest_1sampResult(statistic=0.12183718937633178, pvalue=0.9038683167275986)
Ttest_1samp


Next, do this using the `replicate` function:

```{r}
```


In [13]:
# Not available

How does this compare to the intended false-positive rate of $\alpha=0.05$?

> It's as expected to have false positive rate around 5%

Ok, that was a bit boring. Let's try something more interesting - let's implement a p-value sniffing simulation, in the style of Simons, Nelson, & Simonsohn (2011).

Consider this scenario: you have done an experiment, again with 30 participants (one observation each, just for simplicity). The question is whether the true mean is different from 0. You aren't going to check the p-value every trial, but let's say you run 30 - then if the p-value is within the range p < .25 and p > .05, you optionally run 30 more and add those data, then test again. But if the original p value is < .05, you call it a day, and if the original is > .25, you also stop.  

First, write a function that implements this sampling regime.

```{r}
double.sample <- function () {
}
```

In [22]:
def double_sample(threshold=0.25):
    dist = np.random.normal(size=30)
    ttest_result = stats.ttest_1samp(dist, 0)
    if ttest_result.pvalue < 0.05:
        return True
    elif ttest_result.pvalue < threshold:
        dist_add = np.random.normal(size=30)
        dist = np.concatenate((dist, dist_add))
        ttest_result = stats.ttest_1samp(dist, 0)
        if ttest_result.pvalue < 0.05:
            return True
        else:
            return False


Now call this function 10k times and find out what happens. 

```{r}
```


In [24]:
def test_false_positives(threshold=0.25):
    false_positives = 0
    trial = 10000
    for i in range(trial):
        if double_sample(threshold):
            false_positives += 1

    print(f"False positive rate: {false_positives / trial}")
test_false_positives()

False positive rate: 0.0726



Is there an inflation of false positives? How bad is it?


>The false positive rate is 7% instead of 5%

Now modify this code so that you can investigate this "double the sample" rule in a bit more depth. In the previous question, the researcher doubles the sample only when they think they got "close" to a significant result, i.e. when their not-significant p is less than 0.25. What if the researcher was more optimistic? See what happens in these 3 other scenarios:

* The researcher doubles the sample whenever their pvalue is not significant, but it's less than 0.5.
* The researcher doubles the sample whenever their pvalue is not significant, but it's less than 0.75.
* The research doubles their sample whenever they get ANY pvalue that is not significant.

How do these choices affect the false positive rate?

HINT: Try to do this by making the function `double.sample` take the upper p value as an argument, so that you can pass this through dplyr.

HINT 2: You may need more samples. Find out by looking at how the results change from run to run.

```{r}
```


In [26]:
test_false_positives(0.5)
test_false_positives(0.75)
test_false_positives(1.0)

False positive rate: 0.0792
False positive rate: 0.0851
False positive rate: 0.0882



What do you conclude on the basis of this simulation? How bad is this kind of data-dependent policy?

> By increasing the sample size after looking at the statistics, the false positive rate increases.
> The false positive rate increases as the researchers increase the threshold when they double the sample size
> The increase effect saturates quickly after when the researcher increase the threshold beyond 0.5.